In [ ]:
!curl ipinfo.io

{
  "ip": "34.133.179.123",
  "hostname": "123.179.133.34.bc.googleusercontent.com",
  "city": "Council Bluffs",
  "region": "Iowa",
  "country": "US",
  "loc": "41.2619,-95.8608",
  "org": "AS15169 Google LLC",
  "postal": "51502",
  "timezone": "America/Chicago",
  "readme": "https://ipinfo.io/missingauth"
}

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# get latest version of tfds
![ ! -f "pip_installed" ] && pip install tensorflow-datasets==4.4.0
!touch pip_installed

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1


In [ ]:
!pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [ ]:
# clone repository to contents/TeamSemiSuperCV
!git clone https://github.com/TeamSemiSuperCV/semi-super.git /contents/TeamSemiSuperCV

Cloning into '/contents/TeamSemiSuperCV'...
remote: Enumerating objects: 299, done.
remote: Counting objects: 100% (299/299), done.
remote: Compressing objects: 100% (197/197), done.
remote: Total 299 (delta 164), reused 221 (delta 94), pack-reused 0
Receiving objects: 100% (299/299), 171.97 MiB | 51.73 MiB/s, done.
Resolving deltas: 100% (164/164), done.


In [ ]:
# change dir to TeamSemiSuperCV (repo code)
import os
os.chdir('/contents/TeamSemiSuperCV')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
!git branch

* main


# Stage 3 knowledge distillation with the 1 percent labels model from Stage 2 as the teacher model.

In [ ]:
model_folder = 'dist_nolam_1e5_1pc_rand_run2'

teacher_model_dir = 'gs://simclrv2-sb/reborn_again_1pc_5/saved_model/100'

model_dir = 'gs://simclrv2-sb/'

class simclrCommand():
  def __init__(self, model_dir, model_folder, params={}):
    #default params
    self.params = {'train_mode': "finetune", 
          "mode": "train_then_eval",    
          "resnet_depth": 50, 
          "color_jitter_strength": 0.5,
          'model_dir': model_dir+model_folder,
          "data_dir": model_dir,
          "use_tpu": True}
    # add in or change params
    if len(params) > 0:
      for k,v in params.items():
        self.params[k]=v
  def compile_command(self):
    simclr_command = ['python run.py']
    for k,v in self.params.items():
      simclr_command.append(f'--{k}={v}')
    return (" ").join(simclr_command)
  def run_command(self):
    !{self.compile_command()}

params = {"train_epochs": 1000,
          'checkpoint_epochs': 50,
          "train_batch_size": 128,
          'train_split': 'train', 
          "image_size": 224,   
          'dataset': "xray_reborn",
          "learning_rate": 0.20, 
          "learning_rate_scaling": 'sqrt',
          'weight_decay': 1e-5, 
          "temperature": 0.1,
          "use_blur": False, 
          "area_range_min": 0.5,
          "eval_split": "validation",    
          "eval_per_loop": False,
          'fine_tune_after_block': -1,
          "distill_mode": True,
          "teacher_model_dir": teacher_model_dir,
          "keras_resnet50": True} 
slimsk2 = simclrCommand(model_dir, model_folder, params)
slimsk2.run_command()

slimsk2.params['mode'] = 'eval'
slimsk2.params['eval_split'] = 'test'

slimsk2.run_command()

2021-11-17 03:43:50.562175: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
I1117 03:43:51.991946 139915899733888 dataset_info.py:358] Load dataset info from gs://simclrv2-sb/xray_reborn/1.0.0
I1117 03:43:52.295171 139915899733888 dataset_builder.py:351] Reusing dataset xray_reborn (gs://simclrv2-sb/xray_reborn/1.0.0)
I1117 03:43:52.295498 139915899733888 run.py:619] # train examples: 4708
I1117 03:43:52.295765 139915899733888 run.py:620] # train_steps: 36782
I1117 03:43:52.295823 139915899733888 run.py:621] # eval examples: 524
I1117 03:43:52.295870 139915899733888 run.py:622] # eval steps: 3
I1117 03:43:52.320350 139915899733888 remote.py:237] Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
I1117 03:43:52.320597 139915899733888 tpu_strategy_util.py:61] Deallocate tpu buffers before initializing tpu sy